<a href="https://colab.research.google.com/github/Omar-A-hmed/NAAN-MUDHALVAN-FINAL-PROJECT/blob/main/Personalized%20marketing%20and%20customer%20experience.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from textblob import TextBlob

# 1. Sample Customer Data (interactions + feedback)
data = {
    'User': ['Alice', 'Alice', 'Bob', 'Charlie', 'Charlie', 'Dave'],
    'Product': ['Laptop', 'Phone', 'Laptop', 'Phone', 'Tablet', 'Tablet'],
    'Rating': [5, 3, 4, 4, 5, 2],
    'Feedback': [
        "Love the laptop quality!",
        "Phone is okay.",
        "Good performance.",
        "Battery life is great!",
        "Very responsive and sleek.",
        "Too slow for my work."
    ]
}

df = pd.DataFrame(data)

# 2. Create User-Product Matrix
matrix = df.pivot_table(index='User', columns='Product', values='Rating').fillna(0)

# 3. Similarity Matrix
similarity = pd.DataFrame(
    cosine_similarity(matrix),
    index=matrix.index,
    columns=matrix.index
)

# 4. Recommend Products
def recommend(user, n=2):
    similar_users = similarity[user].sort_values(ascending=False).index[1:]
    owned = df[df['User'] == user]['Product'].tolist()
    scores = {}

    for sim_user in similar_users:
        sim_data = df[df['User'] == sim_user]
        for _, row in sim_data.iterrows():
            if row['Product'] not in owned:
                scores[row['Product']] = scores.get(row['Product'], 0) + row['Rating']
        if len(scores) >= n:
            break

    return sorted(scores.items(), key=lambda x: x[1], reverse=True)[:n]

# 5. Personalized Email Generator
def generate_email(user):
    recs = recommend(user)
    email = f"Hello {user},\n\nBased on your interest, we think you'll love:\n"
    for product, _ in recs:
        email += f"- {product}\n"
    email += "\nVisit our site to explore more personalized deals!\n"
    return email

# 6. Sentiment Analysis
def analyze_sentiment():
    df['Sentiment'] = df['Feedback'].apply(lambda x: TextBlob(x).sentiment.polarity)
    return df[['User', 'Product', 'Feedback', 'Sentiment']]

# 7. Customer Segmentation
def segment_customers():
    sentiment_scores = analyze_sentiment().groupby('User')['Sentiment'].mean()
    segments = sentiment_scores.apply(lambda x: 'Promoter' if x > 0.5 else 'Passive' if x > 0 else 'Detractor')
    return segments

# === DEMO ===
print("=== Recommendations for Alice ===")
print(recommend('Alice'))

print("\n=== Personalized Email ===")
print(generate_email('Alice'))

print("\n=== Sentiment Analysis ===")
print(analyze_sentiment())

print("\n=== Customer Segmentation ===")
print(segment_customers())

=== Recommendations for Alice ===
[('Tablet', 7)]

=== Personalized Email ===
Hello Alice,

Based on your interest, we think you'll love:
- Tablet

Visit our site to explore more personalized deals!


=== Sentiment Analysis ===
      User Product                    Feedback  Sentiment
0    Alice  Laptop    Love the laptop quality!      0.625
1    Alice   Phone              Phone is okay.      0.500
2      Bob  Laptop           Good performance.      0.700
3  Charlie   Phone      Battery life is great!      1.000
4  Charlie  Tablet  Very responsive and sleek.      0.200
5     Dave  Tablet       Too slow for my work.     -0.300

=== Customer Segmentation ===
User
Alice       Promoter
Bob         Promoter
Charlie     Promoter
Dave       Detractor
Name: Sentiment, dtype: object
